In [ ]:
import numpy as np
import cv2
from flask import Flask, render_template, Response, request, redirect, url_for
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import os
from datetime import datetime
import random
from PIL import Image, ImageDraw, ImageFont

# Initialize Flask app
app = Flask(__name__)

# Emotion labels and emoji sets
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 
                  4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
emoji_mapping = {
    0: ['😠', '😡', '😣'],  # Angry
    1: ['🤢', '🤮', '😖'],  # Disgust
    2: ['😨', '😱', '😰'],  # Fear
    3: ['😊', '😂', '🥳'],  # Happy
    4: ['😢', '😭', '🥺'],  # Sad
    5: ['😲', '😯', '🤯'],  # Surprise
    6: ['😐', '😑', '😶']   # Neutral
}

# Load your trained model (update with your exact filename)
model_path = 'model_weights (1).keras'  # Replace with your model's timestamped filename
model = load_model(model_path)
print("Model loaded successfully")

# Preprocess image for prediction
def preprocess_image(image, target_size=(100, 100)):
    image = cv2.resize(image, target_size)
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Predict emotion and recommend one emoji
def predict_emotion(image):
    processed_image = preprocess_image(image)
    prediction = model.predict(processed_image)
    emotion_idx = np.argmax(prediction)
    emotion = emotion_labels[emotion_idx]
    recommended_emoji = random.choice(emoji_mapping[emotion_idx])
    print(f"Predicted: {emotion} {recommended_emoji}")  # Debug output
    return emotion, recommended_emoji

# Webcam feed generator
def gen_frames():
    camera = cv2.VideoCapture(0)
    if not camera.isOpened():
        print("Error: Could not open webcam.")
        return
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    if face_cascade.empty():
        print("Error: Could not load face cascade.")
        return
    # Load a font that supports emojis (Windows default or download one)
    try:
        font_path = "C:/Windows/Fonts/seguiemj.ttf"  # Windows emoji font
        font = ImageFont.truetype(font_path, 30)
    except:
        print("Error: Could not load font. Download an emoji-supporting font (e.g., Noto Emoji).")
        return
    while True:
        success, frame = camera.read()
        if not success:
            print("Error: Could not read frame.")
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        # Convert frame to PIL image for text overlay
        frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(frame_pil)
        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            emotion, emoji = predict_emotion(face)
            # Draw bounding box with OpenCV
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            # Use PIL to draw text and emoji
            display_text = f"{emotion}: {emoji}"
            draw.text((x, y-40), display_text, font=font, fill=(0, 255, 0, 255))
        # Convert back to OpenCV format
        frame = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
    camera.release()

# Routes
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        upload_dir = 'static/uploads'
        os.makedirs(upload_dir, exist_ok=True)
        file_path = os.path.join(upload_dir, f"upload_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg")
        file.save(file_path)
        image = cv2.imread(file_path)
        if image is None:
            return "Error: Could not load image.", 400
        emotion, emoji = predict_emotion(image)
        print(f"Saved image path: {file_path}")
        return render_template('result.html', emotion=emotion, emoji=emoji, image_path=file_path)
    return render_template('upload.html')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)